### This jupyter notebook is to recognize faces on live camera

In [4]:
# importing libraries

from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
import torchvision.transforms as T
from torch.utils.data import DataLoader
from PIL import Image,ImageDraw
import cv2
import time
import os
import numpy as np

In [5]:
# initializing MTCNN and InceptionResnetV1 

mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40) # keep_all=False
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40) # keep_all=True
resnet = InceptionResnetV1(pretrained='vggface2').eval() 

In [6]:
# Read data from folder

dataset = datasets.ImageFolder('poliface_origin') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn0(img, return_prob=True) 
    if face is not None and prob>0.92:
        emb = resnet(face.unsqueeze(0)) 
        embedding_list.append(emb.detach()) 
        name_list.append(idx_to_class[idx])        

# save data
data = [embedding_list, name_list] 
torch.save(data, 'data.pt') # saving data.pt file

In [7]:
transform =T.ToPILImage()

In [8]:
data_test = datasets.ImageFolder('poliface_test')
idx_to_class_test={i:c for c,i in data_test.class_to_idx.items()}
loader_test = DataLoader(data_test,collate_fn=collate_fn)
count=1
name_true=[]
name_test=[]
temp_name=''
for img, idx in loader_test:
    img_cropped_list, prob_list = mtcnn(img, return_prob=True) 
    #print(img_cropped_list[0])
    #print(idx_to_class_test[idx])
    temp_name=idx_to_class_test[idx]
    if img_cropped_list is not None:
        """boxes, _ = mtcnn.detect(img)
        #print(prob_list)
        img_array=np.asarray(img)
        im_arr_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        #temp_frame = img.copy()
        prob_list_list = prob_list.tolist()
        max_value = max(prob_list_list)

        max_index = prob_list_list.index(max_value)
        temp_name = idx_to_class_test"""
        for i,prob in enumerate(prob_list):
            if prob>0.60:
                #temp_frame= embedding_list
                #print('Probability of face:')
                #print(prob)
                
                emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() 
                    
                dist_list = [] # list of matched distances, minimum distance is used to identify the person
                    
                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                min_dist = min(dist_list) # get minumum dist value
                min_dist_idx = dist_list.index(min_dist) # get minumum dist index
                name = name_list[min_dist_idx] # get name corrosponding to minimum dist
        box = boxes[max_index]
        c1,c2 = (int(np.float32(box[0])),int(np.float32(box[1]))),(int(np.float32(box[2])),int(np.float32(box[3])))
        cv2.rectangle(im_arr_bgr,c1,c2,(255,0,0),5)
        img_array=cv2.cvtColor(im_arr_bgr,cv2.COLOR_BGR2RGB)
        im2 = Image.fromarray(img_array)
        if not os.path.exists('poliface_run_max'):
            os.mkdir('poliface_run_max')
        if not os.path.exists('poliface_run_max/'):
            os.mkdir('poliface_run_max/')
        im2 = im2.save("poliface_run_max/{}.jpg".format(count))
        count+=1
        #img_name="poliface_run/{}/{}.jpg".format(name,count)
        #cv2.imwrite(img_name,temp_frame)
        #print(" saved: {}".format(count))
        if min_dist<0.60:
            #temp_frame = cv2.putText(temp_frame, name+' '+str(min_dist), c1, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),1, cv2.LINE_AA)
            print(name+' '+str(min_dist)+' index'+str(count) )
            count+=1
            #print(temp_frame)
            name_test.append(name)
            name_true.append(temp_name)
                    
print(len(name_true),len(name_test)) 
from sklearn.metrics import accuracy_score
print('Accuracy rate:')
print(accuracy_score(name_true, name_test))

ADat_Nacentech 0.5483829379081726 index1
ADat_Nacentech 0.5318140983581543 index2
ADat_Nacentech 0.5164004564285278 index3
ADat_Nacentech 0.5918645858764648 index4
ADat_Nacentech 0.3137890100479126 index5
ADat_Nacentech 0.20562462508678436 index6
ADat_Nacentech 0.5330504179000854 index7
ADat_Nacentech 0.5230152010917664 index8
Duong_guest_NICexpo 0.5596486926078796 index9
AHung_Nacentech 0.46456512808799744 index10
AHung_Nacentech 0.4400879442691803 index11
AHung_Nacentech 0.4041878581047058 index12
AHung_Nacentech 0.43235644698143005 index13
AHung_Nacentech 0.4629649519920349 index14
AHung_Nacentech 0.47770097851753235 index15
AHung_Nacentech 0.521297812461853 index16
AHung_Nacentech 0.5397700071334839 index17
AHung_Nacentech 0.23571129143238068 index18
AHung_Nacentech 0.4477018117904663 index19
AKet_guest_NICexpo 0.5590885877609253 index20
AQuyet_guest_NICexpo 0.5464585423469543 index21
AToan_guest_NICexpo 0.5687696933746338 index22
AToan_guest_NICexpo 0.5723209977149963 index23
AToa

In [9]:
#Testing 
"""ds = datasets.ImageFolder('poliface_test')
idx_to_class_test={i:c for c,i in ds.class_to_idx.items()}
def collate_fn(x):
    return x[0]
loader_test = DataLoader(ds,collate_fn=collate_fn)
name_list_test=[]
embedding_list_test=[]
for img,idx in loader_test:
    face,prob = mtcnn0(img,return_prob= True)
    if face is not None and prob>0.92:
        emb_test=resnet(face.unsqueeze(0))
        embedding_list_test.append(emb_test.detach())
        name_list_test.append(idx_to_class_test[idx])
data_test=[embedding_list_test,name_list_test]
torch.save(data_test,'data_test.pt')
#print(embedding_list_test[1])
embedding_list = np.array(embedding_list)
embedding_list_test = np.array(embedding_list_test)
print(embedding_list.shape, embedding_list_test.shape)
print(len(name_list), len(name_list_test))
print("a")
#print(embedding_list[1])
from sklearn.metrics.pairwise import cosine_similarity

def _most_similarity(embed_vecs, vec, labels):
  sim = cosine_similarity(embed_vecs, vec)
  sim = np.squeeze(sim, axis = 1)
  argmax = np.argsort(sim)[::-1][:1]
  label = [labels[idx] for idx in argmax][0]
  print(label)
  return label
y_preds=[]
probs = [t.detach().numpy() for t in embedding_list]
print(probs[1])
print(probs[0][1])
for i in range(len(embedding_list_test)):
  vec = embedding_list_test[i].reshape(1,-1)
  y_preds=_most_similarity(probs,vec,name_list)
#from sklearn.metrics import accuracy_score
#print(accuracy_score)
"""


'ds = datasets.ImageFolder(\'poliface_test\')\nidx_to_class_test={i:c for c,i in ds.class_to_idx.items()}\ndef collate_fn(x):\n    return x[0]\nloader_test = DataLoader(ds,collate_fn=collate_fn)\nname_list_test=[]\nembedding_list_test=[]\nfor img,idx in loader_test:\n    face,prob = mtcnn0(img,return_prob= True)\n    if face is not None and prob>0.92:\n        emb_test=resnet(face.unsqueeze(0))\n        embedding_list_test.append(emb_test.detach())\n        name_list_test.append(idx_to_class_test[idx])\ndata_test=[embedding_list_test,name_list_test]\ntorch.save(data_test,\'data_test.pt\')\n#print(embedding_list_test[1])\nembedding_list = np.array(embedding_list)\nembedding_list_test = np.array(embedding_list_test)\nprint(embedding_list.shape, embedding_list_test.shape)\nprint(len(name_list), len(name_list_test))\nprint("a")\n#print(embedding_list[1])\nfrom sklearn.metrics.pairwise import cosine_similarity\n\ndef _most_similarity(embed_vecs, vec, labels):\n  sim = cosine_similarity(embe

In [10]:
# Using webcam recognize face

# loading data.pt file
#load_data = torch.load('data.pt') 
#embedding_list = load_data[0] 
#name_list = load_data[1] 

#cam = cv2.VideoCapture(0) 

"""while True:
    ret, frame = cam.read()
    if not ret:
        print("fail to grab frame, try again")
        break
        
    img = Image.fromarray(frame)
    img_cropped_list, prob_list = mtcnn(img, return_prob=True) 
    
    if img_cropped_list is not None:
        boxes, _ = mtcnn.detect(img)
                
        for i, prob in enumerate(prob_list):
            if prob>0.90:
                emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() 
                
                dist_list = [] # list of matched distances, minimum distance is used to identify the person
                
                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                min_dist = min(dist_list) # get minumum dist value
                min_dist_idx = dist_list.index(min_dist) # get minumum dist index
                name = name_list[min_dist_idx] # get name corrosponding to minimum dist
                
                box = boxes[i] 
                
                original_frame = frame.copy() # storing copy of frame before drawing on it
                c1,c2 = (int(np.float32(box[0])),int(np.float32(box[1]))),(int(np.float32(box[2])),int(np.float32(box[3])))
                if min_dist<0.90:
                    frame = cv2.putText(frame, name+' '+str(min_dist), c1, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),1, cv2.LINE_AA)
                    print(name+' '+str(min_dist))
                    print(frame)
                    frame = cv2.rectangle(frame, c1 , c2, (255,0,0), 2)
                    

    cv2.imshow("IMG", frame)
        
    
    k = cv2.waitKey(1)
    if k%256==27: # ESC
        print('Esc pressed, closing...')
        break
        
    elif k%256==32: # space to save image
        print('Enter your name :')
        name = input()
        
        # create directory if not exists
        if not os.path.exists('photos/'+name):
            os.mkdir('photos/'+name)
            
        img_name = "photos/{}/{}.jpg".format(name, int(time.time()))
        cv2.imwrite(img_name, original_frame)
        print(" saved: {}".format(img_name))
        
        
cam.release()
cv2.destroyAllWindows()#
    """

'while True:\n    ret, frame = cam.read()\n    if not ret:\n        print("fail to grab frame, try again")\n        break\n        \n    img = Image.fromarray(frame)\n    img_cropped_list, prob_list = mtcnn(img, return_prob=True) \n    \n    if img_cropped_list is not None:\n        boxes, _ = mtcnn.detect(img)\n                \n        for i, prob in enumerate(prob_list):\n            if prob>0.90:\n                emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() \n                \n                dist_list = [] # list of matched distances, minimum distance is used to identify the person\n                \n                for idx, emb_db in enumerate(embedding_list):\n                    dist = torch.dist(emb, emb_db).item()\n                    dist_list.append(dist)\n\n                min_dist = min(dist_list) # get minumum dist value\n                min_dist_idx = dist_list.index(min_dist) # get minumum dist index\n                name = name_list[min_dist_idx] # get name